In [175]:
import os
from copy import deepcopy

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SplineTransformer,PolynomialFeatures,StandardScaler,MinMaxScaler



from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# MODELS
from sklearn.neighbors import NearestNeighbors

In [176]:
def mean(list):
    return sum(list)/len(list)

def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [2]:
# COLLABORATIVE RECS -- Future application; need users

# Python --> Crab library

In [3]:
# User Input Considerations:

# I'm Feeling Lucky -> Drop a random dot in the housing map

## Price Range: Min - Max

## Property Type

# Minimum Requirements ^^

## State

## City

## Zip Code

## Property Type

## No. of Bedrooms

## No. of Bathrooms

## Square Footage

## Year Built



### NEED TO IDENTIFY WHEN DATA IS SPARSE FOR A CERTAIN AREA AND PRESENT APOLOGY MESSAGE


In [4]:
## CONSIDERATIONS

# INTRINSICALLY ASSIGN EACH PROPERTY PREDICTOR ITS LOCATION

# INTRINSICALLY ASSIGN EACH PROPERTY PREDICTOR ITS PROPERTY TYPE

# RANK THE CHOSEN NEIGHBORS IN A QUEUE, FILTER THE QUEUE BY PRICE RANGE, AND RETURN THE TOP 1-15

In [188]:
aws_bucket=os.getenv('AWS_RE_BUCKET')
aws_key=os.getenv('AWS_RE_BUCKET_KEY')
aws_secret=os.getenv('AWS_RE_BUCKET_SECRET')

date="2023-06-16"
state="CA"
states=['AL',
 'NE',
 'AK',
 'NV',
 'AZ',
 'NH',
 'AR',
 'NJ',
 'CA',
 'NM',
 'CO',
 'NY',
 'CT',
 'NC',
 'DE',
 'ND',
 'DC',
 'OH',
 'FL',
 'OK',
 'GA',
 'OR',
 'HI',
 'PA',
 'ID',
 'PR',
 'IL',
 'RI',
 'IN',
 'SC',
 'IA',
 'SD',
 'KS',
 'TN',
 'KY',
 'TX',
 'LA',
 'UT',
 'ME',
 'VT',
 'MD',
 'VA',
 'MA',
 'VI',
 'MI',
 'WA',
 'MN',
 'WV',
 'MS',
 'WI',
 'MO',
 'WY',
 'MT',
]

In [189]:
# PULLING IN DATA

df_list=[]
for st in states:
    try:
        df = pd.read_parquet(path=aws_bucket+f"/{date}/{st}.parquet",storage_options={"key":aws_key,"secret":aws_secret})
        df_list.append(df)
    except:
        pass
    
df=pd.concat(df_list)

In [190]:
df['PROPERTY TYPE'].unique()

array(['Mobile/Manufactured Home', 'Vacant Land',
       'Single Family Residential', 'Multi-Family (2-4 Unit)',
       'Multi-Family (5+ Unit)', 'Townhouse', 'Ranch', 'Condo/Co-op',
       'Other', 'Parking', 'Timeshare', 'Moorage', 'Unknown', 'None'],
      dtype=object)

In [191]:
df_copy=deepcopy(df)

In [303]:
df=df_copy

In [304]:
# PREPROCESSING
prop_types=['Single Family Residential', 'Mobile/Manufactured Home','Townhouse', 'Multi-Family (2-4 Unit)', 'Condo/Co-op','Multi-Family (5+ Unit)']

df=df.applymap(lambda x: np.nan if x=='nan' else x)

zip_count = df[['ZIP OR POSTAL CODE']].nunique().iloc[0]

#df=df[(df['CITY']=='Houston')&(df['STATE OR PROVINCE']=='TX')] # TEMP LIMITER 1

#df=df[df['PROPERTY TYPE']=='Single Family Residential'] # TEMP LIMITER 2

df=df[df['PROPERTY TYPE'].isin(prop_types)]

df=df[['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE','PRICE','BEDS','BATHS','SQUARE FEET','YEAR BUILT','DAYS ON MARKET','LATITUDE','LONGITUDE']].dropna(axis=0).reset_index(drop=True).apply(lambda row: pd.to_numeric(row,errors='ignore'))

#temp=pd.get_dummies(data=df['PROPERTY TYPE'])

#df_full=pd.concat([df,temp],axis=1)

df_full=df

df_final=df_full.drop(['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE'],axis=1)


In [305]:
df_full

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,LATITUDE,LONGITUDE
0,4900 County Road 15,Maplesville,AL,36750,Mobile/Manufactured Home,179000.0,4.0,2.0,1976.0,2019.0,3.0,32.752520,-86.797908
1,9455 Highway 191,Maplesville,AL,36750,Single Family Residential,650000.0,5.0,3.0,4236.0,2010.0,11.0,32.860645,-86.813504
2,185 County Road 10,Maplesville,AL,36750,Single Family Residential,345000.0,5.0,4.0,2690.0,2002.0,692.0,32.784044,-86.876101
3,231 Birmingham St,Prichard,AL,36610,Single Family Residential,154000.0,3.0,2.0,1624.0,1950.0,7.0,30.738793,-88.076812
4,1417 Saint Claire Ave,Mobile,AL,36610,Multi-Family (2-4 Unit),38000.0,4.0,2.0,1400.0,1950.0,32.0,30.754935,-88.087958
...,...,...,...,...,...,...,...,...,...,...,...,...,...
490983,28 Miles Ave Unit 103,Whitefish,MT,59937,Condo/Co-op,2500000.0,3.0,2.5,2707.0,2023.0,190.0,48.411804,-114.343121
490984,57 Crestwood Dr Unit A,Whitefish,MT,59937,Townhouse,2906000.0,5.0,4.0,2906.0,2022.0,190.0,48.427054,-114.335989
490985,1515 Highway 93 W Unit 326,Whitefish,MT,59937,Condo/Co-op,1999000.0,3.0,3.0,1960.0,2022.0,190.0,48.410027,-114.366198
490986,1515 Highway 93 W Unit 327,Whitefish,MT,59937,Condo/Co-op,1699000.0,2.0,2.0,1698.0,2022.0,190.0,48.410027,-114.366198


## NEED TO DECIDE BETWEEN TIME COST TRADE-OFF FOR JUST TRAINING NEW MODEL AND RUNNING KNEIGHBORS VS USING SAME MODEL APPLYING POST-FIND FILTERS

### FOR NOW, IT SEEMS LIKE IT COULD TAKE A LONG TO ITERATE THROUGH QUEUE TO FIND REQUESTED FILTERS; ESPECIALLY IF REQUESTED LOCATION HAS NO DATA

In [306]:
df_NN=df_final[['BEDS','BATHS','SQUARE FEET','YEAR BUILT']]

In [307]:
df_NN

,BEDS,BATHS,SQUARE FEET,YEAR BUILT
0,4.0,2.0,1976.0,2019.0
1,5.0,3.0,4236.0,2010.0
2,5.0,4.0,2690.0,2002.0
3,3.0,2.0,1624.0,1950.0
4,4.0,2.0,1400.0,1950.0
...,...,...,...,...
490983,3.0,2.5,2707.0,2023.0
490984,5.0,4.0,2906.0,2022.0
490985,3.0,3.0,1960.0,2022.0
490986,2.0,2.0,1698.0,2022.0


In [308]:
# Define the transformations for each column
preprocessor = ColumnTransformer(
transformers=[
    ('minmax_scaler',MinMaxScaler(),['BEDS','BATHS','SQUARE FEET','YEAR BUILT'])
])

# Apply the transformations in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

df_NN=pipeline.fit_transform(df_NN)

In [309]:
input_df = pd.DataFrame({'BEDS':[3],'BATHS':[2],'SQUARE FEET':[1800],'YEAR BUILT':[2000]})
input_df

input_t=pipeline.transform(input_df)
input_t


array([[3.03030303e-02, 3.00000000e-04, 8.64817406e-05, 5.72082380e-01]])

In [310]:

knn=NearestNeighbors(n_neighbors=375000)
knn.fit(df_NN)

distances, indices = knn.kneighbors(input_t)


# CURRENT IMPLEMENTATION PUTS TOO MUCH FOCUS ON EXACT

# E.G., For 2BD, 1.5BA, 1300SQF would choose 4BD,3BA, 1300SQF over 2BD, 1.5BA, 1250SQF
        
        
    
    

In [311]:
df_slice=df_full[df_full['ZIP OR POSTAL CODE']=='77004']

In [312]:
df_slice

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,LATITUDE,LONGITUDE
349423,3866 Wichita St,Houston,TX,77004,Single Family Residential,320000.0,3.0,2.0,2364.0,1970.0,20.0,29.712865,-95.351781
349424,2010 Blodgett St Unit H,Houston,TX,77004,Single Family Residential,379000.0,2.0,2.0,1799.0,2007.0,1.0,29.726870,-95.375840
349425,3412 St Emanuel St,Houston,TX,77004,Townhouse,289900.0,2.0,2.0,1412.0,2005.0,1.0,29.734442,-95.369064
349426,4909 La Branch St,Houston,TX,77004,Single Family Residential,489000.0,3.0,3.5,2415.0,2005.0,1.0,29.727276,-95.381520
349427,5104 Jackson St,Houston,TX,77004,Townhouse,695000.0,3.0,3.5,2942.0,2014.0,1.0,29.725523,-95.381248
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349594,3505 Hutchins St,Houston,TX,77004,Single Family Residential,449000.0,3.0,3.5,2547.0,2015.0,279.0,29.733206,-95.368318
349595,2516 Saint Emanuel St,Houston,TX,77004,Single Family Residential,675000.0,3.0,2.5,2511.0,2022.0,303.0,29.740371,-95.364486
349596,3409 N Macgregor Way,Houston,TX,77004,Single Family Residential,775000.0,3.0,5.0,4784.0,1950.0,550.0,29.711009,-95.365352
349597,2201 Ruth St,Houston,TX,77004,Single Family Residential,777999.0,5.0,4.5,3500.0,1930.0,662.0,29.726790,-95.373365


In [313]:
picks=[]

for i in indices[0]:
    try:
        pick=df_slice.loc[i]
        picks.append(pick)
    except:
        pass
    
    if len(picks)==3:
        break

In [316]:
picks[0]

ADDRESS                          3342 Reeves St
CITY                                    Houston
STATE OR PROVINCE                            TX
ZIP OR POSTAL CODE                        77004
PROPERTY TYPE         Single Family Residential
PRICE                                  250000.0
BEDS                                        3.0
BATHS                                       2.0
SQUARE FEET                              1104.0
YEAR BUILT                               2002.0
DAYS ON MARKET                             49.0
LATITUDE                              29.727252
LONGITUDE                            -95.355255
Name: 349519, dtype: object

In [317]:
picks[0]

ADDRESS                          3342 Reeves St
CITY                                    Houston
STATE OR PROVINCE                            TX
ZIP OR POSTAL CODE                        77004
PROPERTY TYPE         Single Family Residential
PRICE                                  250000.0
BEDS                                        3.0
BATHS                                       2.0
SQUARE FEET                              1104.0
YEAR BUILT                               2002.0
DAYS ON MARKET                             49.0
LATITUDE                              29.727252
LONGITUDE                            -95.355255
Name: 349519, dtype: object

In [318]:
df_full.iloc[349519]

ADDRESS                          3342 Reeves St
CITY                                    Houston
STATE OR PROVINCE                            TX
ZIP OR POSTAL CODE                        77004
PROPERTY TYPE         Single Family Residential
PRICE                                  250000.0
BEDS                                        3.0
BATHS                                       2.0
SQUARE FEET                              1104.0
YEAR BUILT                               2002.0
DAYS ON MARKET                             49.0
LATITUDE                              29.727252
LONGITUDE                            -95.355255
Name: 349519, dtype: object

In [298]:
input_t

array([[3.03030303e-02, 3.00000000e-04, 8.64817406e-05, 5.72082380e-01]])